In [2]:
import shioaji as sj
from dotenv import load_dotenv
import os
import pandas as pd
from datetime import datetime, timedelta
import sqlite3

2025-09-04 10:26:12.204 | WARNING  | importlib._bootstrap:_call_with_frames_removed:241 - Optional: pip install shioaji[speed] or uv add shioaji --extra speed for better performance.


In [3]:
load_dotenv()

def get_api(simulation: bool = True) -> sj.Shioaji:
    api = sj.Shioaji(simulation=simulation)
    api.login(
        api_key=os.environ["API_KEY"],
        secret_key=os.environ["SECRET_KEY"],
    )
    return api
api=get_api()  # 自動登入

Response Code: 0 | Event Code: 0 | Info: host '210.59.255.161:80', hostname '210.59.255.161:80' IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


Response Code: 200 | Event Code: 16 | Info: APISUB/V1/SYS/CONTRACT | Event: Subscribe or Unsubscribe ok


In [38]:
def get_kbar(stock_code) :
    kbars = api.kbars(
        contract=api.Contracts.Stocks[stock_code], 
        start=(datetime.now() - timedelta(days=200)).strftime("%Y-%m-%d"), 
        end=datetime.now().strftime("%Y-%m-%d"), 
    )
    
    return kbars

# 第一款

In [ ]:
stock_code='6515'
kbars=get_kbar(stock_code)

df = pd.DataFrame({**kbars})
df.ts = pd.to_datetime(df.ts)
# df
df = df.groupby(df.ts.dt.date).agg({ "Open": "first", "High": "max", "Low": "min", "Close": "last", "Volume": "sum", "Amount": "sum" })
df.drop(columns=["Open", "High", "Low", 'Amount'], inplace=True)
df['code']=stock_code
df['pct'] = df['Close'].pct_change()
df['6pct'] = df['pct'].rolling(6).sum()

iterm1_1 = (df['6pct']*100 > 32)
iterm1_2 = (df['6pct']*100 > 25) & (df['Close'] - df['Close'].shift(5) >=50)

df['notice']= iterm1_1 | iterm1_2 
df['pre_punished'] = (df['notice'].rolling(2).sum()==2) & (df['notice'].rolling(3).sum()!=3)

df['target_pct_1(%)'] = (32-df['pct'].rolling(5).sum()*100).where(df['pre_punished'])
df['target_pct_2_v1(%)'] = (25-df['pct'].rolling(5).sum()*100).where(df['pre_punished'])
df['target_pct_2_v2($)'] = (df['Close'].shift(4) + 50).where(df['pre_punished'])

df['target_info1_1'] = df.apply(
    lambda row: '一定處置' if pd.notna(row['target_pct_1(%)']) and row['target_pct_1(%)'] < -10
                else f"漲超過{row['target_pct_1(%)']:.2f}%以上,價位:{row['Close']*(1+row['target_pct_1(%)']/100):.2f}"
                if pd.notna(row['target_pct_1(%)']) else None,
    axis=1
)

df['target_info1_2'] = df.apply(
    lambda row: '一定處置' if (pd.notna(row['target_pct_2_v1(%)'])) and (row['target_pct_2_v1(%)'] < -10) and (row['target_pct_2_v2($)'] < row['Close']*0.9 )
                else f"漲超過{row['target_pct_2_v1(%)']:.2f}%以上,價位:{max(row['target_pct_2_v2($)'],row['Close']*(1+row['target_pct_2_v1(%)']/100)):.2f}"
                if pd.notna(row['target_pct_2_v1(%)']) else None,
    axis=1
)

df = df[df['pre_punished']][['target_info1_1','target_info1_2']]
df

,target_info1_1,target_info1_2
ts,,
2025-09-02,"漲超過0.88%以上,價位:1548.46","漲超過-6.12%以上,價位:1300.00"


# 第二款

In [ ]:
stock_code='3167'
kbars=get_kbar(stock_code)


,Close,code,30pct,60pct,90pct,notice,pre_punished,target_pct2_1(%),target_pct2_2(%),target_pct2_3(%),target_pct_2($),target_info2(%),target_info2($)
ts,,,,,,,,,,,,,
2025-07-11,149.0,3167,0.740654,1.302937,0.586794,True,True,32.959641,3.899848,103.322818,149.0,"漲超過3.90%以上,價位:154.81",價位:149.00
2025-07-14,154.0,3167,0.726457,1.336874,0.619348,True,True,43.017329,-20.406504,95.645678,154.0,"漲超過-20.41%以上,價位:122.57",價位:154.00
2025-07-15,160.0,3167,0.630989,1.601626,0.707577,True,True,40.000000,-23.968254,90.328738,160.0,"漲超過-23.97%以上,價位:121.65",價位:160.00
2025-07-18,161.0,3167,0.330579,1.421053,0.856978,True,True,60.606061,-7.463127,75.154994,161.0,"漲超過-7.46%以上,價位:148.98",價位:161.00
2025-07-21,166.5,3167,0.441558,1.455752,0.911596,True,True,62.396694,-10.606936,66.620209,166.5,"漲超過-10.61%以上,價位:148.84",價位:166.50
2025-07-22,170.0,3167,0.404959,1.456647,0.974448,True,True,57.142857,-6.439499,64.597701,170.0,"漲超過-6.44%以上,價位:159.05",價位:170.00
2025-07-23,172.0,3167,0.445378,1.392211,0.977011,True,True,67.692308,-10.559441,64.100228,172.0,"漲超過-10.56%以上,價位:153.84",價位:172.00
2025-07-28,169.5,3167,0.313953,1.456522,0.952765,True,True,63.306452,-14.588745,68.906426,169.5,"漲超過-14.59%以上,價位:144.77",價位:169.50
2025-08-01,187.0,3167,0.551867,1.506702,1.144495,True,True,46.090535,-12.542153,47.981859,187.0,"漲超過-12.54%以上,價位:163.55",價位:187.00


In [80]:
df = pd.DataFrame({**kbars})
df.ts = pd.to_datetime(df.ts)

# df
df = df.groupby(df.ts.dt.date).agg({ "Open": "first", "High": "max", "Low": "min", "Close": "last", "Volume": "sum", "Amount": "sum" })
df.drop(columns=["Open", "High", "Low", 'Amount','Volume'], inplace=True)
df['code']=stock_code
df['30pct'] = df['Close'].pct_change(29)
df['60pct'] = df['Close'].pct_change(59)
df['90pct'] = df['Close'].pct_change(89)

iterm2_1 = (df['30pct'] * 100 >100) 
iterm2_2 = (df['60pct'] * 100 >130) 
iterm2_3 = (df['90pct'] * 100 >160) 

item2_all = (df['Close']>df["Close"].shift(1))

df['notice'] = (iterm2_1 | iterm2_2 | iterm2_3) & item2_all
df['pre_punished'] = df['notice']

df['target_pct2_1(%)'] = (100-df['Close'].pct_change(28)*100).where(df['pre_punished'])
df['target_pct2_2(%)'] = (130-df['Close'].pct_change(58)*100).where(df['pre_punished'])
df['target_pct2_3(%)'] = (160-df['Close'].pct_change(88)*100).where(df['pre_punished'])

df['target_pct_2($)'] = df['Close'].where(df['pre_punished'])

df['target_info2(%)'] = df.apply(
    lambda row: '起訖漲幅一定達標' if pd.notna(row['target_pct2_1(%)']) and max(row['target_pct2_1(%)'],row['target_pct2_2(%)'],row['target_pct2_3(%)']) < -10
                else f"漲超過{min(row['target_pct2_1(%)'],row['target_pct2_2(%)'],row['target_pct2_3(%)']):.2f}%以上"
                if pd.notna(row['target_pct2_1(%)']) else None,
    axis=1
)

df['target_info2($)'] = df.apply(
    lambda row: f"價位:{row['target_pct_2($)']:.2f}"
                if pd.notna(row['target_pct_2($)']) else None,
    axis=1
)

# df[df["notice"]]['notice']
df[df['pre_punished']][['target_info2(%)','target_info2($)']]
# df[df['pre_punished']]

,target_info2(%),target_info2($)
ts,,
2025-07-11,漲超過3.90%以上,價位:149.00
2025-07-14,漲超過-20.41%以上,價位:154.00
2025-07-15,漲超過-23.97%以上,價位:160.00
2025-07-18,漲超過-7.46%以上,價位:161.00
2025-07-21,漲超過-10.61%以上,價位:166.50
2025-07-22,漲超過-6.44%以上,價位:170.00
2025-07-23,漲超過-10.56%以上,價位:172.00
2025-07-28,漲超過-14.59%以上,價位:169.50
2025-08-01,漲超過-12.54%以上,價位:187.00


# 第三款(target_infos未寫)

In [44]:
stock_code='1717'
kbars=get_kbar(stock_code)
df = pd.DataFrame({**kbars})
df.ts = pd.to_datetime(df.ts)

df = df.groupby(df.ts.dt.date).agg({ "Open": "first", "High": "max", "Low": "min", "Close": "last", "Volume": "sum", "Amount": "sum" })
df.drop(columns=["Open", "High", "Low", 'Amount'], inplace=True)
df['code']=stock_code
df['pct'] = df['Close'].pct_change()
df['6pct'] = df['pct'].rolling(6).sum()
df['60volume'] = df['Volume'].rolling(60).mean()

iterm3 = (df['6pct'] * 100 > 25) & (df['Volume'] > df['60volume'] * 5 )

df['notice']= iterm3
# df['iterm1_1'] = (df['6pct']*100 > 32).rolling(2).sum() >= 2
# df['iterm1_2'] = ((df['6pct']*100 > 25).rolling(2).sum() >= 2) & (df['Close'] - df['Close'].shift(5) >=50)
# df['target_pct_1(%)'] = (32-df['pct'].rolling(5).sum()*100).where(df['iterm1_1'])
# df['target_pct_2_v1(%)'] = (25-df['pct'].rolling(5).sum()*100).where(df['iterm1_2'])
# df['target_pct_2_v2($)'] = (df['Close'].shift(5) + 50).where(df['iterm1_2'])
# df['target_info1_1'] = df.apply(
#     lambda row: '一定處置' if pd.notna(row['target_pct_1(%)']) and row['target_pct_1(%)'] < -10
#                 else f"漲超過{row['target_pct_1(%)']:.2f}%以上,價位:{row['Close']*(1+row['target_pct_1(%)']/100):.2f}"
#                 if pd.notna(row['target_pct_1(%)']) else None,
#     axis=1
# )

# df['target_info1_2'] = df.apply(
#     lambda row: '一定處置' if (pd.notna(row['target_pct_2_v1(%)'])) and (row['target_pct_2_v1(%)'] < -10) and (row['target_pct_2_v2($)'] < row['Close']*0.9 )
#                 else f"漲超過{row['target_pct_2_v1(%)']:.2f}%以上,價位:{max(row['target_pct_2_v2($)'],row['Close']*(1+row['target_pct_2_v1(%)']/100)):.2f}"
#                 if pd.notna(row['target_pct_2_v1(%)']) else None,
#     axis=1
# )
# # df[['Close','6pct','iterm1_1','iterm1_2','target_pct_2_v1(%)','target_pct_2_v2($)','target_info1_1','target_info1_2']]
# df=df[df['iterm1_1'] | df['iterm1_2']][['code','target_info1_1','target_info1_2']]
df[df["notice"]]['notice']

ts
2025-08-15    True
2025-08-18    True
2025-08-19    True
Name: notice, dtype: bool

# 第四款(週轉率待修)

In [ ]:
stock_code='1717'
kbars=get_kbar(stock_code)
df = pd.DataFrame({**kbars})
df.ts = pd.to_datetime(df.ts)

df = df.groupby(df.ts.dt.date).agg({ "Open": "first", "High": "max", "Low": "min", "Close": "last", "Volume": "sum", "Amount": "sum" })
df.drop(columns=["Open", "High", "Low", 'Amount'], inplace=True)
df['code']=stock_code
df['pct'] = df['Close'].pct_change()
df['6pct'] = df['pct'].rolling(6).sum()
df['60volume'] = df['Volume'].rolling(60).mean()

iterm3 = (df['6pct'] * 100 > 25) & (df['Volume'] > df['60volume'] * 5 )

df['notice']= iterm3
# df['iterm1_1'] = (df['6pct']*100 > 32).rolling(2).sum() >= 2
# df['iterm1_2'] = ((df['6pct']*100 > 25).rolling(2).sum() >= 2) & (df['Close'] - df['Close'].shift(5) >=50)
# df['target_pct_1(%)'] = (32-df['pct'].rolling(5).sum()*100).where(df['iterm1_1'])
# df['target_pct_2_v1(%)'] = (25-df['pct'].rolling(5).sum()*100).where(df['iterm1_2'])
# df['target_pct_2_v2($)'] = (df['Close'].shift(5) + 50).where(df['iterm1_2'])
# df['target_info1_1'] = df.apply(
#     lambda row: '一定處置' if pd.notna(row['target_pct_1(%)']) and row['target_pct_1(%)'] < -10
#                 else f"漲超過{row['target_pct_1(%)']:.2f}%以上,價位:{row['Close']*(1+row['target_pct_1(%)']/100):.2f}"
#                 if pd.notna(row['target_pct_1(%)']) else None,
#     axis=1
# )

# df['target_info1_2'] = df.apply(
#     lambda row: '一定處置' if (pd.notna(row['target_pct_2_v1(%)'])) and (row['target_pct_2_v1(%)'] < -10) and (row['target_pct_2_v2($)'] < row['Close']*0.9 )
#                 else f"漲超過{row['target_pct_2_v1(%)']:.2f}%以上,價位:{max(row['target_pct_2_v2($)'],row['Close']*(1+row['target_pct_2_v1(%)']/100)):.2f}"
#                 if pd.notna(row['target_pct_2_v1(%)']) else None,
#     axis=1
# )
# # df[['Close','6pct','iterm1_1','iterm1_2','target_pct_2_v1(%)','target_pct_2_v2($)','target_info1_1','target_info1_2']]
# df=df[df['iterm1_1'] | df['iterm1_2']][['code','target_info1_1','target_info1_2']]
df[df["notice"]]['notice']

# 第五款(券商市占待修)

In [ ]:
stock_code='1717'
kbars=get_kbar(stock_code)
df = pd.DataFrame({**kbars})
df.ts = pd.to_datetime(df.ts)

df = df.groupby(df.ts.dt.date).agg({ "Open": "first", "High": "max", "Low": "min", "Close": "last", "Volume": "sum", "Amount": "sum" })
df.drop(columns=["Open", "High", "Low", 'Amount'], inplace=True)
df['code']=stock_code
df['pct'] = df['Close'].pct_change()
df['6pct'] = df['pct'].rolling(6).sum()

iterm5 = (df['6pct'] * 100 > 25) & (df['Volume'] > 500 )

df['notice']= iterm5
# df['iterm1_1'] = (df['6pct']*100 > 32).rolling(2).sum() >= 2
# df['iterm1_2'] = ((df['6pct']*100 > 25).rolling(2).sum() >= 2) & (df['Close'] - df['Close'].shift(5) >=50)
# df['target_pct_1(%)'] = (32-df['pct'].rolling(5).sum()*100).where(df['iterm1_1'])
# df['target_pct_2_v1(%)'] = (25-df['pct'].rolling(5).sum()*100).where(df['iterm1_2'])
# df['target_pct_2_v2($)'] = (df['Close'].shift(5) + 50).where(df['iterm1_2'])
# df['target_info1_1'] = df.apply(
#     lambda row: '一定處置' if pd.notna(row['target_pct_1(%)']) and row['target_pct_1(%)'] < -10
#                 else f"漲超過{row['target_pct_1(%)']:.2f}%以上,價位:{row['Close']*(1+row['target_pct_1(%)']/100):.2f}"
#                 if pd.notna(row['target_pct_1(%)']) else None,
#     axis=1
# )

# df['target_info1_2'] = df.apply(
#     lambda row: '一定處置' if (pd.notna(row['target_pct_2_v1(%)'])) and (row['target_pct_2_v1(%)'] < -10) and (row['target_pct_2_v2($)'] < row['Close']*0.9 )
#                 else f"漲超過{row['target_pct_2_v1(%)']:.2f}%以上,價位:{max(row['target_pct_2_v2($)'],row['Close']*(1+row['target_pct_2_v1(%)']/100)):.2f}"
#                 if pd.notna(row['target_pct_2_v1(%)']) else None,
#     axis=1
# )
# # df[['Close','6pct','iterm1_1','iterm1_2','target_pct_2_v1(%)','target_pct_2_v2($)','target_info1_1','target_info1_2']]
# df=df[df['iterm1_1'] | df['iterm1_2']][['code','target_info1_1','target_info1_2']]
df[df["notice"]]['notice']

# 第六款(估值類待修)

In [ ]:
stock_code='1717'
kbars=get_kbar(stock_code)
df = pd.DataFrame({**kbars})
df.ts = pd.to_datetime(df.ts)

df = df.groupby(df.ts.dt.date).agg({ "Open": "first", "High": "max", "Low": "min", "Close": "last", "Volume": "sum", "Amount": "sum" })
df.drop(columns=["Open", "High", "Low", 'Amount'], inplace=True)
df['code']=stock_code
df['pct'] = df['Close'].pct_change()
df['6pct'] = df['pct'].rolling(6).sum()

iterm5 = (df['6pct'] * 100 > 25) & (df['Volume'] > 500 )

df['notice']= iterm5
# df['iterm1_1'] = (df['6pct']*100 > 32).rolling(2).sum() >= 2
# df['iterm1_2'] = ((df['6pct']*100 > 25).rolling(2).sum() >= 2) & (df['Close'] - df['Close'].shift(5) >=50)
# df['target_pct_1(%)'] = (32-df['pct'].rolling(5).sum()*100).where(df['iterm1_1'])
# df['target_pct_2_v1(%)'] = (25-df['pct'].rolling(5).sum()*100).where(df['iterm1_2'])
# df['target_pct_2_v2($)'] = (df['Close'].shift(5) + 50).where(df['iterm1_2'])
# df['target_info1_1'] = df.apply(
#     lambda row: '一定處置' if pd.notna(row['target_pct_1(%)']) and row['target_pct_1(%)'] < -10
#                 else f"漲超過{row['target_pct_1(%)']:.2f}%以上,價位:{row['Close']*(1+row['target_pct_1(%)']/100):.2f}"
#                 if pd.notna(row['target_pct_1(%)']) else None,
#     axis=1
# )

# df['target_info1_2'] = df.apply(
#     lambda row: '一定處置' if (pd.notna(row['target_pct_2_v1(%)'])) and (row['target_pct_2_v1(%)'] < -10) and (row['target_pct_2_v2($)'] < row['Close']*0.9 )
#                 else f"漲超過{row['target_pct_2_v1(%)']:.2f}%以上,價位:{max(row['target_pct_2_v2($)'],row['Close']*(1+row['target_pct_2_v1(%)']/100)):.2f}"
#                 if pd.notna(row['target_pct_2_v1(%)']) else None,
#     axis=1
# )
# # df[['Close','6pct','iterm1_1','iterm1_2','target_pct_2_v1(%)','target_pct_2_v2($)','target_info1_1','target_info1_2']]
# df=df[df['iterm1_1'] | df['iterm1_2']][['code','target_info1_1','target_info1_2']]
df[df["notice"]]['notice']

# 第七款(券資比待修)

In [ ]:
stock_code='1717'
kbars=get_kbar(stock_code)
df = pd.DataFrame({**kbars})
df.ts = pd.to_datetime(df.ts)

df = df.groupby(df.ts.dt.date).agg({ "Open": "first", "High": "max", "Low": "min", "Close": "last", "Volume": "sum", "Amount": "sum" })
df.drop(columns=["Open", "High", "Low", 'Amount'], inplace=True)
df['code']=stock_code
df['pct'] = df['Close'].pct_change()
df['6pct'] = df['pct'].rolling(6).sum()

iterm5 = (df['6pct'] * 100 > 25) & (df['Volume'] > 500 )

df['notice']= iterm5
# df['iterm1_1'] = (df['6pct']*100 > 32).rolling(2).sum() >= 2
# df['iterm1_2'] = ((df['6pct']*100 > 25).rolling(2).sum() >= 2) & (df['Close'] - df['Close'].shift(5) >=50)
# df['target_pct_1(%)'] = (32-df['pct'].rolling(5).sum()*100).where(df['iterm1_1'])
# df['target_pct_2_v1(%)'] = (25-df['pct'].rolling(5).sum()*100).where(df['iterm1_2'])
# df['target_pct_2_v2($)'] = (df['Close'].shift(5) + 50).where(df['iterm1_2'])
# df['target_info1_1'] = df.apply(
#     lambda row: '一定處置' if pd.notna(row['target_pct_1(%)']) and row['target_pct_1(%)'] < -10
#                 else f"漲超過{row['target_pct_1(%)']:.2f}%以上,價位:{row['Close']*(1+row['target_pct_1(%)']/100):.2f}"
#                 if pd.notna(row['target_pct_1(%)']) else None,
#     axis=1
# )

# df['target_info1_2'] = df.apply(
#     lambda row: '一定處置' if (pd.notna(row['target_pct_2_v1(%)'])) and (row['target_pct_2_v1(%)'] < -10) and (row['target_pct_2_v2($)'] < row['Close']*0.9 )
#                 else f"漲超過{row['target_pct_2_v1(%)']:.2f}%以上,價位:{max(row['target_pct_2_v2($)'],row['Close']*(1+row['target_pct_2_v1(%)']/100)):.2f}"
#                 if pd.notna(row['target_pct_2_v1(%)']) else None,
#     axis=1
# )
# # df[['Close','6pct','iterm1_1','iterm1_2','target_pct_2_v1(%)','target_pct_2_v2($)','target_info1_1','target_info1_2']]
# df=df[df['iterm1_1'] | df['iterm1_2']][['code','target_info1_1','target_info1_2']]
df[df["notice"]]['notice']

# 第八款(TDR待修)

In [ ]:
stock_code='1717'
kbars=get_kbar(stock_code)
df = pd.DataFrame({**kbars})
df.ts = pd.to_datetime(df.ts)

df = df.groupby(df.ts.dt.date).agg({ "Open": "first", "High": "max", "Low": "min", "Close": "last", "Volume": "sum", "Amount": "sum" })
df.drop(columns=["Open", "High", "Low", 'Amount'], inplace=True)
df['code']=stock_code
df['pct'] = df['Close'].pct_change()
df['6pct'] = df['pct'].rolling(6).sum()

iterm5 = (df['6pct'] * 100 > 25) & (df['Volume'] > 500 )

df['notice']= iterm5
# df['iterm1_1'] = (df['6pct']*100 > 32).rolling(2).sum() >= 2
# df['iterm1_2'] = ((df['6pct']*100 > 25).rolling(2).sum() >= 2) & (df['Close'] - df['Close'].shift(5) >=50)
# df['target_pct_1(%)'] = (32-df['pct'].rolling(5).sum()*100).where(df['iterm1_1'])
# df['target_pct_2_v1(%)'] = (25-df['pct'].rolling(5).sum()*100).where(df['iterm1_2'])
# df['target_pct_2_v2($)'] = (df['Close'].shift(5) + 50).where(df['iterm1_2'])
# df['target_info1_1'] = df.apply(
#     lambda row: '一定處置' if pd.notna(row['target_pct_1(%)']) and row['target_pct_1(%)'] < -10
#                 else f"漲超過{row['target_pct_1(%)']:.2f}%以上,價位:{row['Close']*(1+row['target_pct_1(%)']/100):.2f}"
#                 if pd.notna(row['target_pct_1(%)']) else None,
#     axis=1
# )

# df['target_info1_2'] = df.apply(
#     lambda row: '一定處置' if (pd.notna(row['target_pct_2_v1(%)'])) and (row['target_pct_2_v1(%)'] < -10) and (row['target_pct_2_v2($)'] < row['Close']*0.9 )
#                 else f"漲超過{row['target_pct_2_v1(%)']:.2f}%以上,價位:{max(row['target_pct_2_v2($)'],row['Close']*(1+row['target_pct_2_v1(%)']/100)):.2f}"
#                 if pd.notna(row['target_pct_2_v1(%)']) else None,
#     axis=1
# )
# # df[['Close','6pct','iterm1_1','iterm1_2','target_pct_2_v1(%)','target_pct_2_v2($)','target_info1_1','target_info1_2']]
# df=df[df['iterm1_1'] | df['iterm1_2']][['code','target_info1_1','target_info1_2']]
df[df["notice"]]['notice']

# DB

In [ ]:
conn = sqlite3.connect("target_info.db")
df.to_sql("target_info", conn, if_exists="replace", index=True)
conn.close()

target_info= pd.read_sql("SELECT * FROM target_info", sqlite3.connect("target_info.db"))
target_info